In [1]:
#Import Libraries
import sys
import os
import sqlite3 as lite

from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import json
import pickle


pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)


print(os.getcwd())
print(sys.version)

C:\Users\domin\OneDrive\Twitter_Sent_Analysis\testing_playground
3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]


In [2]:
traindf = pd.read_json("Training_Data.json")
traindf['tweet_clean_string'] = [' , '.join(z).strip() for z in traindf['tweet']]  
traindf['tweet_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['tweet']]       

testdf = pd.read_json("C:/Users/domin/OneDrive/TextMining/Goodfiles/test2_public1a.json") 
testdf['tweet_clean_string'] = [' , '.join(z).strip() for z in testdf['tweet']]
testdf['tweet_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in testdf['tweet']]

In [3]:
#build training corpus
corpustr = traindf['tweet_string']
vectorizertr = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)
tfidftr=vectorizertr.fit_transform(corpustr)

#build testing corpus
corpusts = testdf['tweet_string']
vectorizerts = TfidfVectorizer(stop_words='english')
tfidfts=vectorizertr.transform(corpusts)

In [4]:
predictors_tr = tfidftr

targets_tr = traindf['sentiment']

predictors_ts = tfidfts

In [5]:
#classifier = LinearSVC(C=0.80, penalty="l2", dual=False)
parameters = {'C':[1, 10]}
#clf = LinearSVC()
clf = LogisticRegression()

In [6]:
classifier = GridSearchCV(clf, parameters)

classifier=classifier.fit(predictors_tr,targets_tr)

c:\users\domin\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\domin\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
#Predict the sentiment of test data
# predictions=classifier.predict(predictors_ts)
# testdf['sentiment'] = predictions

In [10]:
#Write test sentiment to CSV for delivery
# testdf[['id' , 'sentiment' ]].to_csv("boccaleri.submission2.csv")

In [12]:
#save the model
filename = "sentiment_model.sav"
pickle.dump(classifier, open(filename,'wb'))